**Install Packages**

In [10]:
!pip install torch trochvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [11]:
!pip install laserembeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:
import pandas as pd
import random as rd
import json
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors


In [13]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# Monolingual Embedding

In [14]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['به ملازمان سلطان که رساند این دعا را که به شکر پادشاهی ز نظر مران گدا را ز رقیب دیوسیرت به خدای خود پناهم مگر آن شهاب ثاقب مددی دهد خدا را',
     'هزار و سیصدتا سلام'],
    lang='fa')  # lang is only used for tokenization

In [15]:
print(embeddings.shape)
print(embeddings)
%cd "/content/"

(2, 1024)
[[ 0.01072003  0.00345858  0.00174004 ...  0.02418301  0.01499301
   0.01545107]
 [ 0.0038445   0.00011095  0.00274064 ...  0.0077541   0.00159712
  -0.00810177]]
/content


# Multilingual Embedding

In [16]:
#Example
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

**Input data set**

read input data set and creat the input table

In [17]:
data_set_root = "/content/drive/MyDrive/Text_enrich/datasets/"
input_data = pd.read_csv(data_set_root + "input_text/input_text.csv")

In [18]:
columns = ["input_text" , "vector"]
input_table = pd.DataFrame([], columns=columns)
for sample in rd.sample(list(pd.unique(input_data["input_text"])),100):
  input_table = input_table.append({'input_text':sample,'vector':laser.embed_sentences(sample, 'fa')}, ignore_index = True)

In [19]:
input_table

,input_text,vector
0,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,"[[0.006814094, -0.00036874664, 0.0015877598, 0..."
1,باید اشتباهات دوستان را تذکر داد,"[[0.005557922, -0.0005153683, 0.001958137, 0.0..."
2,باید در انتخاب محلی که خود را قرار می‌دهیم دقت...,"[[0.012048804, -7.184042e-05, 0.04076763, 0.01..."
3,آخرت را باید بر دنیا ترجیح داد.,"[[-0.0002303459, -5.3586803e-08, -8.24661e-06,..."
4,نباید رابطه خویشاوندی را قطع کرد,"[[0.0045186304, -0.00018031245, 0.00044726304,..."
...,...,...
95,ویژگی‌های خوب قابل کسب کردن هستند,"[[0.011362986, -6.4524655e-05, 0.0019776826, 0..."
96,بنده دیگران شدن کار اشتباهی است,"[[0.04188226, -0.00043034053, 0.016514702, 0.0..."
97,باید از فریب دادن دیگران پرهیز کرد,"[[0.029407907, -0.00079821964, 0.0029948354, 0..."
98,نباید عیب دیگران را بر ملا کنیم,"[[0.030206732, -8.934332e-05, 0.0016241245, 0...."


# Output data set embedding




English qoutes

In [20]:
columns = ["output_text" , "vector"]
English_qoute_table = pd.DataFrame([], columns=columns)
f = open(data_set_root +'quotes/quotes.jsonl')
quotes = [json.loads(jline) for jline in f.read().splitlines()]
for i in range(len(quotes)):
  if len(quotes[i]["quote"]) < 210:
    English_qoute_table = English_qoute_table.append({'output_text':quotes[i]["quote"][1:-2],'vector':laser.embed_sentences(quotes[i]["quote"][1:-2], 'en')}, ignore_index = True)

f.close()

In [21]:
English_qoute_table

,output_text,vector
0,Be yourself; everyone else is already taken,"[[0.03283381, 0.0025794504, 0.022685742, 0.008..."
1,"I'm selfish, impatient and a little insecure. ...","[[0.018681081, 0.00092175644, 0.0030920352, 0...."
2,Two things are infinite: the universe and huma...,"[[0.005447657, -3.156499e-05, 0.0062513533, 0...."
3,"So many books, so little time","[[0.008000842, 0.0071800463, 0.0014852672, 0.0..."
4,A room without books is like a body without a ...,"[[0.0024867677, -0.00081185653, -0.0034946627,..."
...,...,...
2042,"Stepping onto a brand-new path is difficult, b...","[[0.0055467826, -0.00013396252, 0.00923926, 0...."
2043,Morality is simply the attitude we adopt towar...,"[[0.0024499279, -0.00011392398, 0.011446702, 0..."
2044,"In life, finding a voice is speaking and livin...","[[0.018904364, 0.013504211, -0.0012334686, 0.0..."
2045,"Winter is the time for comfort, for good food ...","[[0.011106329, -1.8142506e-05, -0.0011179153, ..."


In [22]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(English_qoute_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(English_qoute_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + English_qoute_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید دانشمندان را شناخت و از آن‌ها قدردانی کرد => If books could have more, give more, be more, show more, they would still need readers who bring to them sound and smell and light and all the rest that canâ€™t be in books.	The book needs you
____________________________________________________________________________________________________
باید اشتباهات دوستان را تذکر داد => But it is the nature of stars to cross, and never was Shakespeare more wrong than when he has Cassius note, â€˜The fault, dear Brutus, is not in our stars / But in ourselves
____________________________________________________________________________________________________
باید در انتخاب محلی که خود را قرار می‌دهیم دقت کنیم. => There is a saying in Tibetan, 'Tragedy should be utilized as a source of strength.'No matter what sort of difficulties, how painful experience is, if we lose our hope, that's our real disaster
________________________________________________________________________________________________

Nahj Al-Balaqa

In [23]:
columns = ["output_text" , "vector"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/Nahj Al-Balaqa.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
for line in Lines:
    splited = line.split("\t")
    if len(splited[1]) < 300: 
      Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar')}, ignore_index = True)
file1.close()


In [24]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(Nahj_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(Nahj_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + Nahj_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید دانشمندان را شناخت و از آن‌ها قدردانی کرد => وَ قَالَ ع إِنّ لِلقُلُوبِ شَهوَةً وَ إِقبَالًا وَ إِدبَاراً فَأتُوهَا مِن قِبَلِ شَهوَتِهَا وَ إِقبَالِهَا فَإِنّ القَلبَ إِذَا أُکرِهَ عمَیِ‌َ

____________________________________________________________________________________________________
باید اشتباهات دوستان را تذکر داد => وَ سَأَلَهُ رَجُلٌ أَن یُعَرّفَهُ الإِیمَانَ فَقَالَ ع إِذَا کَانَ الغَدُ فأَتنِیِ حَتّی أُخبِرَکَ عَلَی أَسمَاعِ النّاسِ فَإِن نَسِیتَ مقَاَلتَیِ حَفِظَهَا عَلَیکَ غَیرُکَ فَإِنّ الکَلَامَ کَالشّارِدَةِ یَنقُفُهَا هَذَا وَ یُخطِئُهَا هَذَا

____________________________________________________________________________________________________
باید در انتخاب محلی که خود را قرار می‌دهیم دقت کنیم. => وَ قَالَ ع لَتَعطِفَنّ الدّنیَا عَلَینَا بَعدَ شِمَاسِهَا عَطفَ الضّرُوسِ عَلَی وَلَدِهَا وَ تَلَا عَقِیبَ ذَلِکَوَ نُرِیدُ أَن نَمُنّ عَلَی الّذِینَ استُضعِفُوا فِی الأَرضِ وَ نَجعَلَهُم أَئِمّةً وَ نَجعَلَهُمُ الوارِثِینَ

___________________________________________

Yek-Beyti

In [25]:
columns = ["output_text" , "vector"]
tak_beit_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'poems/yek-beiti.txt', 'r')
Lines = file1.readlines()
for line in Lines:
    print(line)
    tak_beit_table = tak_beit_table.append({'output_text':line,'vector':laser.embed_sentences(line, 'fa')}, ignore_index = True)
file1.close()
     


در سخن گفتن خطای جاهلان پیدا شود/ تیر کج چون از کمان بیرون دود رسوا شود

ترا دیدم و یوسف را شنیدم/ شنیدن کی بود مانند دیدن

رزق را روزی رسان پر میدهد/ بی مگس هرگز نماند عنکبوت

دل ز پر گفتن بمیرد در بدن/ گر چه گفتارش بود در عدن

امشب تو را به خوبی نسبت به ماه کردم/ تو خوب تر ز ماهی من اشتباه کردم

به همان قدر که چشم تو پر از زیباییست/ بی تو دنیای من ای دوست پر از تنهاییست

در این فکرم که خواهی ماند با من مهربان یا نه/ به من کم میکنی لطفی که داری این زمان یا نه

به صد سال یک دوست آید به دست/ به یک روز توان کرد دشمن به شصت

طاعت آن نیست که بر خاک نهی پیشانی/ صدق پیش آر که ابلیس بسی کرد سجود

ز گفتار بیهوده جز رنج نیست/ چو خاموشی اندر جهان گنج نیست

بزرگی سراسر به گفتار نیست/ دو صد گفته چون نیم کردار نیست

مباش درپی آزار و هرچه خواهی کن/  که در شریعت ما غیر از این گناهی نیست

یک عمر دور و تنها تنها به جرم اینکه/ او سرسپرده میخواست من دل سپرده بودم

قناعت کن اگر در آرزوی گنج قارونی/  گدای خویش باش ار طالب ملک سلیمانی

میرود کز ما جدا گردد ولی/ جان و دل با اوست هر جا میرود

به زیورها بیارای

In [26]:
tak_beit_table

,output_text,vector
0,در سخن گفتن خطای جاهلان پیدا شود/ تیر کج چون ا...,"[[0.0051233326, 0.0021247042, -8.56197e-05, 0...."
1,ترا دیدم و یوسف را شنیدم/ شنیدن کی بود مانند د...,"[[-0.00043157677, -5.2949763e-06, -0.000173926..."
2,رزق را روزی رسان پر میدهد/ بی مگس هرگز نماند ع...,"[[-2.6380663e-05, -2.2061666e-08, -0.000192369..."
3,دل ز پر گفتن بمیرد در بدن/ گر چه گفتارش بود در...,"[[0.010522119, -5.5991004e-05, -0.001832582, 0..."
4,امشب تو را به خوبی نسبت به ماه کردم/ تو خوب تر...,"[[0.0068011666, 0.0012913884, 0.001053597, 0.0..."
...,...,...
344,در این عالم از هرچه بایستنیست/ نکوتر ز فرزند ش...,"[[0.0038596916, 0.009699946, 0.001407909, 0.01..."
345,در جوانی پاک بودن شیوه پیغمبریست/ ورنه هر گبری...,"[[0.002896377, 0.005819695, 0.0035321082, 0.02..."
346,درست نیست دو رنگی میان ظاهر و باطن/ بگو شکسته ...,"[[0.00969487, 0.00028041055, 0.0045149554, 0.0..."
347,در غربت مرگ بیم تنهایی نیست/ یاران عزیز آن طرف...,"[[0.00076011557, 0.00011960601, -5.3893527e-06..."


*Evaluate* 

In [27]:


for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(tak_beit_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(tak_beit_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + tak_beit_table["output_text"].iloc[index] )  
  print("_"*100) 
     


باید دانشمندان را شناخت و از آن‌ها قدردانی کرد => آنجا که جهل مایه تمکین و سروریست/ باید به روز مردم دانا گریستن

____________________________________________________________________________________________________
باید اشتباهات دوستان را تذکر داد => از دشمنان برند شکایت به دوستان/ چون دوست دشمن است شکایت کجا بریم

____________________________________________________________________________________________________
باید در انتخاب محلی که خود را قرار می‌دهیم دقت کنیم. => مثل تنهایی یک کوچه در این شهر شلوغ/ دلخوش آمدن و رفتن یک رهگذرم

____________________________________________________________________________________________________
آخرت را باید بر دنیا ترجیح داد. => چون روزی و عمر بیش و کمک نتوان کرد/ دل را به کم و بیش دژم نتوان کرد

____________________________________________________________________________________________________
نباید رابطه خویشاوندی را قطع کرد => از سخن حال خرابم نشد اصلاح پذیر / همچ ویرانه که از گنج خود آباد نگشت

_________________________________________________

غزلیات حافظ







In [28]:


columns = ["output_text" , "vector"]
hafez_table = pd.DataFrame([], columns=columns)
file1 = open( data_set_root +'hafez/hafez-qazals.txt', 'r')
Lines = file1.readlines()
flag = False
qazals = []
for line in Lines:
    # line = line.replace("/","")
    # print(line)
    qazal = r".*غزل.*"
    qazal_reg = f"{qazal}"    
    if(re.match(qazal_reg, line) and len(line)<20):
      print(line)
      input =  '\r\n'.join(qazals)
      # print("_"*100)
      hafez_table = hafez_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa')}, ignore_index = True)
      qazals = []
    else:
      if(len(line)>5): 
        qazals.append(line.strip())

    
file1.close()

     


﻿غزل    ۱

غزل    ۲

غزل    ۳

غزل   ۴

غزل   ۵

غزل    ۶

غزل    ۷

غزل    ۸

غزل    ۹

غزل    ۱۰

غزل    ۱۰

غزل    ۱۲

غزل    ۱۳

غزل    ۱۴

غزل   ۱۵

غزل    ۱۶

غزل    ۱۷

غزل   ۱۸

غزل    ۱۹

غزل    ۲۰

غزل    ۲۱

غزل    ۲۲

غزل    ۲۳

غزل    ۲۴

غزل    ۲۵

غزل    ۲۶

غزل    ۲۷

غزل    ۲۸

غزل    ۲۹

غزل    ۳۰

غزل    ۳۱

غزل    ۳۲

غزل    ۳۳

غزل   ۳۴

غزل    ۳۵

غزل    ۳۶

غزل    ۳۷

غزل    ۳۸

غزل    ۳۹

غزل    ۴۰

غزل    ۴۱

غزل    ۴۲

غزل    ۴۳

غزل    ۴۴

غزل    ۴۵

غزل    ۴۶

غزل    ۴۷

غزل    ۴۸

غزل    ۴۹

غزل    ۵۰

غزل    ۵۱

غزل    ۵۲

غزل    ۵۳

غزل    ۵۴

غزل    ۵۵

غزل    ۵۶

غزل    ۵۷

غزل    ۵۸

غزل    ۵۹

غزل    ۶۰

غزل    ۶۱

غزل    ۶۲

غزل    ۶۳

غزل    ۶۴

غزل    ۶۵

غزل    ۶۶

غزل    ۶۷

غزل    ۶۸

غزل    ۶۹

غزل    ۷۰

غزل    ۷۱

غزل    ۷۲

غزل    ۷۳

غزل    ۷۴

غزل    ۷۵

غزل    ۷۶

غزل    ۷۷

غزل    ۷۸

غزل    ۷۹

غزل    ۸۰

غزل    ۸۱

غزل    ۸۲

غزل    ۸۳

غزل    ۸۴

غزل    ۸۵

غزل    ۸۶

غزل    ۸۷

غزل    ۸۸

غزل    ۸۹

غزل    ۹۰

غزل    ۹۱

غزل    ۹۲

غ

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(hafez_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(hafez_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> " + " غزل شماره " + str(index))  
  print(hafez_table["output_text"].iloc[index] )
  print("_"*100) 

غزلیات شمس

In [30]:
moulavi_qazals_path = data_set_root + "moulavi/divane-e-shams.txt"
moulavi_qazals_pattern = re.compile(r'\n?\s*\d+\s*?\n')

def read_divan_shams():
    with open(moulavi_qazals_path, encoding='utf-8') as f:
        text = f.read()
        
    qazals_arr = moulavi_qazals_pattern.split(text)
    return qazals_arr[1:]


In [ ]:
# columns = ["output_text" , "vector"]
# molavi_table = pd.DataFrame([], columns=columns)

# for i in range(0, 500):
#   input = read_divan_shams()[i][0:500]
#   print(i)
#   molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa')}, ignore_index = True)

In [ ]:
# for k in range(len(input_table)):
#   max = 0.0
#   index = 0

#   for i in range(len(molavi_table)-1):
#     image = np.array(input_table["vector"].iloc[k])@np.array(molavi_table["vector"].iloc[i+1]).T
#     if(image > max):
#       max = image
#       index = i+1

#   print(input_table["input_text"].iloc[k] + "  ======>>>> ")  
#   print(molavi_table["output_text"].iloc[index] )
#   print("_"*100) 

**Selecting Top N Output for each input**

In [128]:


def create_pair_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  for i in range(len(input_table)):
    vector = []
    for j in range(len(output_table)): 
      input_vector = input_table.iloc[i]['vector']
      output_vector = output_table.iloc[j]['vector']
      vector.append(abs(input_vector[0] @ output_vector[0].T))
    vector = np.array(vector)
    indexes = vector.argsort()[-k:]
    
    for index in indexes:
      output_data_source_index = str(output_data_source) + " => " + str(index + 1)
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_index,"input_text":input_table.iloc[i]['input_text'],"input_vector":input_table.iloc[i]['vector'],"output_text":output_table.iloc[index]['output_text'] , "output_vector":output_table.iloc[index]['vector']}, ignore_index = True)
  
  return pair_table

     


In [129]:
create_pair_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)

,model,input_text,output_data_source,input_vector,output_text,output_vector
0,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 228,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",استاد کائنات که این جهان ساخت/ مقصود عشق بود ا...,"[[0.004336831, 0.00013955297, -0.0003136543, 0..."
1,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 182,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",آنجا که جهل مایه تمکین و سروریست/ باید به روز ...,"[[0.00044180136, 0.022231705, 0.010233658, 0.0..."
2,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 159,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",آنجا که جهل مایه تمکین و سروریست/ باید به روز ...,"[[0.00038259168, 0.022177326, 0.0001327116, 0...."
3,LASER,باید اشتباهات دوستان را تذکر داد,تک ابیات => 74,"[[0.005557922, -0.0005153683, 0.001958137, 0.0...",گر ما خطا کنیم، عطای تو بی حد است/ نومیدی از ع...,"[[0.0062807994, 0.045346238, -0.00028602427, 0..."
4,LASER,باید اشتباهات دوستان را تذکر داد,تک ابیات => 247,"[[0.005557922, -0.0005153683, 0.001958137, 0.0...",قانع به یک استخوان چو کرکس بودن/ به که طفیل خو...,"[[0.007646164, 0.002562769, 0.00035679742, 0.0..."
...,...,...,...,...,...,...
295,LASER,نباید عیب دیگران را بر ملا کنیم,تک ابیات => 223,"[[0.030206732, -8.934332e-05, 0.0016241245, 0....",از سخن حال خرابم نشد اصلاح پذیر / همچ ویرانه ک...,"[[-0.00015700786, 0.003103194, 0.0009970183, 0..."
296,LASER,نباید عیب دیگران را بر ملا کنیم,تک ابیات => 12,"[[0.030206732, -8.934332e-05, 0.0016241245, 0....",مباش درپی آزار و هرچه خواهی کن/ که در شریعت م...,"[[0.0152184265, 0.001441613, -0.0030149333, 0...."
297,LASER,کلمات ناخواسته و حالت چهره می‌توان آنچه که در ...,تک ابیات => 10,"[[0.0050401744, -0.0007278131, 0.0041338326, 0...",ز گفتار بیهوده جز رنج نیست/ چو خاموشی اندر جها...,"[[0.018255733, -0.000117060714, -0.0030013558,..."
298,LASER,کلمات ناخواسته و حالت چهره می‌توان آنچه که در ...,تک ابیات => 201,"[[0.0050401744, -0.0007278131, 0.0041338326, 0...",قفس شکسته و بال و پرم گشاده و ترسم/ خدا نکرده ...,"[[0.001582338, -2.3269453e-05, 0.011936702, 0...."


Cosine KNN

In [133]:
def create_pair_cosine_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  model = NearestNeighbors(n_neighbors=k,
                          metric='cosine',
                          algorithm='brute',
                          n_jobs=-1)
  x = []
  # Convert out put to array
  for i in range (0, len(tak_beit_table)):
    x.append(tak_beit_table["vector"][i][0])

  model.fit(x)

  for n in range(0, len(input_table)):
    v = input_table["vector"][n]
    for j in range(0, k):
      output_data_source_index = str(model.kneighbors(v, k, return_distance=False)[0][j])
      output_data_source_compelete = str(output_data_source) + " => " + output_data_source_index
      input_text = input_table['input_text'][n]
      output_text = tak_beit_table["output_text"][model.kneighbors(v, k, return_distance=False)[0][j]]
      output_vector = tak_beit_table["vector"][model.kneighbors(v, k, return_distance=False)[0][j]]
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_compelete,"input_text":input_text,"input_vector":v ,"output_text":output_text , "output_vector":output_vector}, ignore_index = True)
  return pair_table


In [134]:
create_pair_cosine_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)

,model,input_text,output_data_source,input_vector,output_text,output_vector
0,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 334,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",خرم آن دانش‌پژوهی کز نخستین روز درس/گوش دل با ...,"[[0.00988015, -0.00032167707, 0.0021140303, 0...."
1,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 123,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",گرفتم آنکه جهانی به یاد ما بودند/ دگر چه فایده...,"[[0.004501415, -0.0003056976, 0.0026356124, 0...."
2,LASER,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,تک ابیات => 181,"[[0.006814094, -0.00036874664, 0.0015877598, 0...",آنجا که جهل مایه تمکین و سروریست/ باید به روز ...,"[[0.00044180136, 0.022231705, 0.010233658, 0.0..."
3,LASER,باید اشتباهات دوستان را تذکر داد,تک ابیات => 259,"[[0.005557922, -0.0005153683, 0.001958137, 0.0...",بس که می‌ترسم از جداییها/ می‌گریزم از آشناییها\n,"[[0.014105117, 0.0021152387, 0.0036042496, 0.0..."
4,LASER,باید اشتباهات دوستان را تذکر داد,تک ابیات => 209,"[[0.005557922, -0.0005153683, 0.001958137, 0.0...",آن روز قبول درگه دوست شوی/ کز رد و قبول خلق پر...,"[[0.0079541365, -0.00020688152, -0.000340794, ..."
...,...,...,...,...,...,...
295,LASER,نباید عیب دیگران را بر ملا کنیم,تک ابیات => 344,"[[0.030206732, -8.934332e-05, 0.0016241245, 0....",در این عالم از هرچه بایستنیست/ نکوتر ز فرزند ش...,"[[0.0038596916, 0.009699946, 0.001407909, 0.01..."
296,LASER,نباید عیب دیگران را بر ملا کنیم,تک ابیات => 261,"[[0.030206732, -8.934332e-05, 0.0016241245, 0....",بنازم به بزم محبت که آنجا / گدایی به شاهی مقاب...,"[[0.0055188187, 0.004446601, 0.014521809, 0.02..."
297,LASER,کلمات ناخواسته و حالت چهره می‌توان آنچه که در ...,تک ابیات => 120,"[[0.0050401744, -0.0007278131, 0.0041338326, 0...",آه افسوس و سرشک گرم و داغ حسرت است/ آنچه از عم...,"[[0.004289172, -0.0003962088, 0.0030683796, 0...."
298,LASER,کلمات ناخواسته و حالت چهره می‌توان آنچه که در ...,تک ابیات => 234,"[[0.0050401744, -0.0007278131, 0.0041338326, 0...",ای دل غمین مشو که جهانت به کام نیست/ کآسایش زم...,"[[0.007296567, -0.00025173798, 0.0015049173, 0..."
